In [1]:
from pitchr import data
from pitchr import xml_parser
from pitchr import harmony_maker
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
DATA_PATH = "../dataset/_xml_scores"
score_files = os.listdir(DATA_PATH)

In [2]:
# gets list of all melody and harmony dfs
all_melody_dfs, all_harmony_dfs = xml_parser.get_all_data()
print(len(all_melody_dfs), len(all_harmony_dfs))

210 210


In [3]:
# Converting dfs to numpy arrays
all_melody_np = []
all_harmony_np = []
for df in all_melody_dfs:
    df = df[['Pitch Number', 'Pitch Interval']]
    temp_np = df.to_numpy()
    all_melody_np.append(temp_np)
for df in all_harmony_dfs:
    df = df[['Pitch Number', 'Pitch Interval']]
    temp_np = df.to_numpy()
    all_harmony_np.append(temp_np)


# normalize harmony sizes
all_harmony_np = xml_parser.prepare_harmony(all_harmony_np)

In [4]:
# convert dataframes to numpy arrays
all_melody_np = np.asarray(all_melody_np)
all_harmony_np = np.asarray(all_harmony_np)

In [6]:
# convert rest notes to -50 and ints to floats
for array in all_melody_np:
    for note in array:
        for i in range(0, 2):
            if type(note[i]) == str:
                note[i] = float(-50)
                np.float32
            elif type(note[i]) == int:
                note[i] = float(note[i])
for array in all_harmony_np:
    for note in array:
        for i in range(0, 2):
            if type(note[i]) == str:
                note[i] = float(-50)
            elif type(note[i]) == int:
                note[i] = float(note[i])


print(all_melody_np.shape)
print(all_harmony_np.shape)

(210, 50, 2)
(210, 50, 2)


In [7]:
X = tf.convert_to_tensor(all_melody_np, dtype=tf.float32)
Y = tf.convert_to_tensor(all_harmony_np, dtype=tf.float32)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(all_melody_np, all_harmony_np, test_size=0.2)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.1)
print(x_train.shape, y_train.shape)
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
x_validation = tf.convert_to_tensor(x_validation, dtype=tf.float32)
y_validation = tf.convert_to_tensor(y_validation, dtype=tf.float32)
normalized_x_train = x_train/50
normalized_y_train = y_train/50
normalized_x_test = x_test/50
normalized_y_test = y_test/50
normalized_x_validation = x_validation/50
normalized_y_validation = y_validation/50

(151, 50, 2) (151, 50, 2)


In [9]:
print(normalized_x_train.shape)
print(normalized_y_train.shape)

(151, 50, 2)
(151, 50, 2)


In [49]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)

model = Sequential()

# 2 LSTM layers
#model.add(LSTM(units=300, batch_input_shape=(1, 50, 2), return_sequences=True))
model.add(LSTM(300, input_shape=(50, 2), return_sequences=True))
model.add(Dropout(0.2))

# Dense layers
model.add(Dense(300, activation="tanh"))
model.add(Dense(200, activation="tanh"))
model.add(Dense(100, activation="tanh"))

# Output Layer
model.add(Dense(2))
model.compile(optimizer=optimizer, metrics=['accuracy'], loss='mse')
trained_model = model.fit(normalized_x_train, normalized_y_train, epochs=100, 
                          validation_data=(normalized_x_validation, normalized_y_validation), batch_size = 2)
print(model.summary())

Epoch 1/5
76/76 [==============================] - 4s 55ms/step - loss: 0.2969 - accuracy: 0.5726 - val_loss: 0.1705 - val_accuracy: 0.6459
Epoch 2/5
76/76 [==============================] - 4s 52ms/step - loss: 0.1335 - accuracy: 0.6432 - val_loss: 0.1146 - val_accuracy: 0.7835
Epoch 3/5
76/76 [==============================] - 4s 54ms/step - loss: 0.1393 - accuracy: 0.6360 - val_loss: 0.1271 - val_accuracy: 0.7835
Epoch 4/5
76/76 [==============================] - 4s 57ms/step - loss: 0.1333 - accuracy: 0.6477 - val_loss: 0.1170 - val_accuracy: 0.7835
Epoch 5/5
76/76 [==============================] - 5s 65ms/step - loss: 0.1314 - accuracy: 0.6483 - val_loss: 0.1159 - val_accuracy: 0.7835
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 50, 300)           363600    
_________________________________________________________________
dropout_28 (D

In [50]:
# testing the output
mariah = all_melody_np[10]
print(mariah.shape)
mariah = mariah.reshape(1, 50, 2)
test_input = tf.convert_to_tensor(mariah, dtype=tf.float32)
#test_input = test_input/50
print("test_input.shape:", test_input.shape)
test_output = model.predict(test_input, verbose=1)
print(test_output.shape)

# this output was normalized so we multiply by 50
test_output = test_output*50
test_output = test_output.reshape(50, 2)
print(test_output)

(50, 2)
test_input.shape: (1, 50, 2)
1/1 [==============================] - 0s 996us/step
(1, 50, 2)
[[-20.7715      -1.8181287 ]
 [-31.755537    -4.428517  ]
 [-18.73362     17.042746  ]
 [-13.471755     5.102666  ]
 [-24.46158     -6.0613036 ]
 [ -0.07491522   5.5913544 ]
 [  3.1826544    1.4109924 ]
 [-11.528749    -8.535046  ]
 [-19.193014     0.9145676 ]
 [-30.317423    -6.0680165 ]
 [-21.435806    19.145056  ]
 [-31.684633     2.8526516 ]
 [ -4.9166775    4.433795  ]
 [-14.270409    -6.463093  ]
 [ -3.8768048    1.474433  ]
 [-15.997289    -7.204376  ]
 [-21.026308    -6.1206207 ]
 [ -4.9971814    1.0238616 ]
 [-12.843636    -8.933651  ]
 [-20.227102     0.6916015 ]
 [  1.5744683    3.2763987 ]
 [  2.9389362   -1.4780966 ]
 [ -3.858241    -1.3884411 ]
 [ -7.4779263   -9.068396  ]
 [ -4.814264     5.3215375 ]
 [-13.953817    -4.8119235 ]
 [-21.631676    -6.3554807 ]
 [ -5.2376804    2.0314693 ]
 [-11.760865    -8.811786  ]
 [-18.208767     0.83822745]
 [-21.728218   -10.471206  ]


In [ ]:
model.save('saved_model/my_model') 

In [51]:
model.save('saved_model/my_model') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model\assets
